In [ ]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
from ids_pullback import *
%matplotlib inline

In [ ]:
### how to make from permatation matrix readable array
def perm_array(A):
    p_a = np.array((A.shape[0]))
    p_a = np.argmax(A, axis = 1)
    return p_a

In [ ]:
def form_permute(C, j, ind):
    C[ind],C[j]=C[j],C[ind]
    return()  

In [ ]:
n = 60 #nmber of sampling points   
deg = 9 #Degree of polynomiapaired plu Last Checkpoint: 22 minutes ago (unsaved changes)l          
interval = [-3, 3]  # interval of x
x = np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1

In [ ]:
### generating matrix with special structure
b = Gen_1D_coupled(p_size, x, debug=True)

In [ ]:
p,l,u,q,inf = pluq_ids(b)

In [ ]:
### maxvol preprocessing
A_init = np.dot(perm_matrix(p),np.dot(b,perm_matrix(q)))
ids_init = A_init[:A_init.shape[1]]
temp_init = np.dot(A_init,np.linalg.inv(ids_init))

In [ ]:
### here we are checking all pairs under Identity matrix and try to find 2x2 block with det>1+tol
tol = 0.01
A = np.copy(A_init)
ids = np.copy(ids_init)
temp = np.copy(temp_init)
n = temp.shape[0]
m = temp.shape[1]
curr_det = np.abs(np.linalg.det(ids))
max_det = 0.0
Fl = True
P = np.arange(n)
index = np.zeros((2), dtype = int)
iters = 0
max_iters = 150

while Fl and (iters < max_iters) :
    max_det = 0.99
    for k in range(m,n,2):
            pair = temp[k:k+2]
            for j in range(0,m,2):
                if np.abs(np.linalg.det(pair[:,j:j+2])) > max_det :
                    max_det = np.abs(np.linalg.det(pair[:,j:j+2]))
                    index[0] = k
                    index[1] = j

    if (max_det) > (1 + tol):
        #Forming new permutation array
        form_permute(P,index[1],index[0])
        form_permute(P, index[1]+1, index[0]+1)
        
        ### Recalculating with new rows position
        A_temp = np.copy(A)
        A = np.dot(perm_matrix(P), A_temp)
        ids = A[:A.shape[1]]
        if np.abs(np.linalg.det(ids)) > curr_det :
            curr_det = np.abs(np.linalg.det(ids))
            print ('det increased')
        
        temp = np.dot(A,np.linalg.inv(ids))
        iters += 1
    else:
        Fl = False                      
        

In [ ]:
### compare det of initial dominant submatrix with another one after blocked maxvol iterations
print np.linalg.det(ids_init)
print np.linalg.det(ids)
iters

In [ ]:
mat_test = np.array([ 
    [50,  0 , 0,0,0, 1],
    [0, 0, 30,  0 ,0, 3],
    [0,0, 0, 20,  0 , 11],
    [0, 40,  0 ,0,0, 2],
    [0,0, 0,0,  10,  12],
    [0,0,0, 0,0, 50]
], dtype=float)
print (mat_test)
print ()

In [ ]:
p,l,u,q, info = pluq_ids(mat_test, debug=True)

In [ ]:
mat_test_3 = np.array([ 
    [20,0, 0, 2, 0 , 11],
    [0, 10,  0 ,0,0.043, 2],    
    [0,  0 , 30,0,15, 1],
    [0, 0, 0,  10 ,0, 0],
    [0,0, 20,0,  90,  12],
    [0,0,0, 0,0, 1900]
], dtype=float)
print (mat_test_3.astype(int))
print ()

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_3, debug=True)

# RGV 2
### Test for linear dependency. 1st thought about pullbacks

In [ ]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

## Check, that matrix is good

In [ ]:
np.linalg.matrix_rank(mat_test)

In [ ]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

In [ ]:
p,l,u,q, info = pluq_ids_pull(mat_test, debug=True)
print info

# Another Error 
### That was about columns swap. 
If piv[0] == j, piv[1] != j+1

In [ ]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_5, debug=True)

## Yet another bug
About feature of maxvol's performance. It returns error if we have a long pair consisted from more then 2 elements each with rank < 2

In [ ]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_6, debug=True)

## Yet another bug ver. 2.0: Division by Zero
### This and below: again about columns swap. Solved

In [ ]:
mat_test_7 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_7, debug=True)

## Yet another bug ver. 3.0: Division by Zero

In [ ]:
mat_test_8 = np.array([[ -5,  -9, -13,  -2,  19,  11],
       [ 14, -17,  -8,   2, -12,   0],
       [ 17, -19,  -3,   6, -19,   1],
       [ 19,   0, -16, -19,  19,  18],
       [ -5,   2,   1,   4, -17,   1],
       [  4,   2,   7,   9, -10, -15],
       [ 19,  -9,   0, -12,   0,  -6],
       [-10,  15,  12,   2,   2,  13],
       [ 16,   0,   3, -16,  16, -15],
       [-15, -14,   9,   8,   0,  17]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_8, debug=True)